In [18]:
import pandas as pd
import numpy as np

function Num_Stab_Approx

RM=6
penalty= 7
normalised= 1
X(1:T-1,:)= trial 4 datax
y(1:T-1,:)=trial 4 datay
X is generated assuming d=1

In [19]:
test = pd.read_csv("trial4_x.csv", encoding = "ISO-8859-1")
add=test.columns.tolist()
test.loc[-1] = add 
test.index = test.index + 1  
test= test.sort_index()
test.columns=["blah","blah2","blah3"]
x = np.copy(test)

test = pd.read_csv("trial4_y.csv", encoding = "ISO-8859-1")
add=test.columns.tolist()
test.loc[-1] = add 
test.index = test.index + 1  
test= test.sort_index()
test.columns=["blah"]
y= np.copy(test)
y[0] = float(y[0])
x[0] = [1., 48.29917764394355828, 1.]

Function begin

In [25]:
RM = 6
penalty= 7 
normalised= 1

Function body

In [24]:
T = x.shape[0]
n = x.shape[1]
N = y.shape[1] #Compute the number of columns in Y

In [ ]:
if normalised == 1 or RM >= 5:
    
    X1 = np.divide(
    (x[:,1:n] - np.matmul(np.ones((T, 1)), x[:,1:n].mean(axis=0).reshape(1, n-1))),
    np.matmul(np.ones((T, 1)), np.array([np.std(x[:,i], ddof=1) for i in range(1,n)]).reshape(1,n-1))
    ) 
    #unfortunately np.std seems cannot return an array for each column, therefore I was forced to use a list comprehension here
    Y1 = np.divide(
    (y - np.ones((T,1))*np.mean(y)),
    np.matmul(np.ones((T,1)), np.std(y, ddof=1).reshape(1,1))
    )
    
    n1 = n-1 # Number of coefficients in a regression with normalized data is reduced by 1 (no intercept)
else:
    X1 = x
    Y1 = Y          # Leave Y without changes
    n1 = n

In [50]:
X1 = np.divide(
    (x[:,1:n] - np.matmul(np.ones((T, 1)), x[:,1:n].mean(axis=0).reshape(1, n-1))),
    np.matmul(np.ones((T, 1)), np.array([np.std(x[:,i], ddof=1) for i in range(1,n)]).reshape(1,n-1))
    ) 
    #unfortunately np.std seems cannot return an array for each column, therefore I was forced to use a list comprehension here
Y1 = np.divide(
    (y - np.ones((T,1))*np.mean(y)),
    np.matmul(np.ones((T,1)), np.std(y, ddof=1).reshape(1,1))
    )
n1 = n-1 # Number of coefficients in a regression with normalized data is reduced by 1 (no intercept)

In [54]:
# 3. Regression methods


In [53]:
fuck2 = 